In [14]:
import random
import numpy as np
from numpy.random import *
import matplotlib.pyplot as plt
import scipy.stats
from scipy.special import expit
import pandas as pd


np.random.seed(1234)


In [15]:
import random

random.seed(1234)

In [16]:
#データ数
num_data = 200

#年齢
x_1 = randint(15, 76, num_data)

#性別(0 を女性、　１を男性)
x_2 = randint(0, 2, num_data)

In [17]:
# ノイズの生成
e_z = randn(num_data)

#シグモイド関数に入れる部分
z_base = x_1 + (1 - x_2)*10 - 40 + 5*e_z

#シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビcmを見たかどうかの変数
Z = np.array([])


for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1 - z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

In [20]:
#ノイズの生成
e_y = randn(num_data)

Y =  -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

In [21]:
df = pd.DataFrame({
    "年齢": x_1,
    "性別":x_2,
    "CMを見た":Z,
    "購入量":Y
})

df.head()

,年齢,性別,CMを見た,購入量
0,62,0,1.0,30.586705
1,34,0,0.0,60.877433
2,53,1,1.0,81.543435
3,68,1,1.0,59.043083
4,27,1,0.0,97.663514


In [25]:
print(df[df["CMを見た"] == 1.0].mean())
print("-"*15)
print(df[df["CMを見た"] == 0.0].mean())

年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      48.752893
dtype: float64
---------------
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      67.300857
dtype: float64


In [26]:
#因果推論を回帰分析で求める

from sklearn.linear_model import LinearRegression

#説明変数
X = df[["年齢", "性別", "CMを見た"]]


#被説明変数
y = df["購入量"]

reg = LinearRegression().fit(X, y)


#回帰した結果の係数を出力
print("係数　：", reg.coef_)

係数　： [-1.10773564 30.20665594 14.00403939]


In [27]:
# 傾向スコアを求める
from sklearn.linear_model import LogisticRegression


#説明変数
X = df[["年齢", "性別"]]

#被説明変数
Z = df["CMを見た"]


reg = LogisticRegression().fit(X, Z)

print("係数beta : ", reg.coef_)
print("係数alpha : ", reg.intercept_ )

係数beta :  [[ 0.10562765 -1.38263933]]
係数alpha :  [-3.37146523]


In [28]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])
print("-"*15)
print(Z[0:5])

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]
---------------
0    1.0
1    0.0
2    1.0
3    1.0
4    0.0
Name: CMを見た, dtype: float64


In [29]:
ATE_i = Y/Z_pre[:, 1]*Z - Y/Z_pre[:,0]*(1-Z)
ATE = 1/len(Y)*ATE_i.sum()
print("推定したATE", ATE)

推定したATE 14.511429725609634
